In [1]:
from arcgis.gis import GIS
from azure.storage.queue import QueueService
import json
import logging
from base64 import b64encode

import sys
sys.path.append('../src')
sys.path.append('../')

from reach_tools import ReachFeatureLayer
import config

# set up logger
logger = logging.getLogger('azure')
logger.setLevel('INFO')

# create the connection to the GIS to get access
gis = GIS(username=config.arcgis_username, password=config.arcgis_password)
logger.info(f'Connected to GIS at {gis.url}')

# create a layer instance to query and get all existing features
lyr_centroid = ReachFeatureLayer(config.url_reach_centroid, gis)
logger.info(f'Successfully created {lyr_centroid.properties.name} layer.')

# creat a connection to the Azure Queue
queue = QueueService(connection_string=config.azure_queue_conn_str)
logger.info(f'Successfully connected to the {config.azure_queue_update_name} Queue')

# query the feature service to get all existing features - notably the reach ids
reach_id_fs = lyr_centroid.query(out_fields='reach_id', return_geometry=False)
logger.info(f'Retrieved {len(reach_id_fs.features)} from the {lyr_centroid.properties.name} layer.')

# iterate the features and load up the queue with each respective feature
for idx, feature in enumerate(reach_id_fs.features):
    
    # convert the feature json to a string
    feature_str = json.dumps(feature.as_dict)
    feature_b64 = b64encode(feature_str.encode('utf-8'))
    
    # push the feature to the queue
    queue.put_message(config.azure_queue_update_name, feature_b64.decode('utf-8'))
             
logger.info(f'Successfully loaded {idx+1} features into the {config.azure_queue_update_name} Queue.')

KeyboardInterrupt: 